In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Dropout, BatchNormalization, Conv1D, AvgPool1D, Flatten
from keras.callbacks import EarlyStopping
from keras.regularizers import L1L2
from sklearn.model_selection import StratifiedKFold
import keras

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### load data

In [3]:
X = np.load('data/abide/postprocessed_timeseries.npy')
Xcorr = np.load('data/abide/postprocessed_corrmats.npy')

In [4]:
# auxiliary inputs
df = pd.read_csv('data/abide/postprocessed_metadata.csv')
age_and_sex_df_centered = df[['AGE_AT_SCAN', 'SEX']] - [15, 1.5]
age_and_sex_df_normed = age_and_sex_df_centered / [10, 1]
offline_train = np.array(age_and_sex_df_normed)

In [5]:
# labels
Y = np.array(df['DX_GROUP'] - 1)

### inspect shapes

In [6]:
Y.shape

(741,)

In [7]:
X.shape

(741, 295, 200)

In [8]:
Xcorr.shape

(741, 19900)

In [9]:
offline_train.shape

(741, 2)

### model params for LSTM (some common to linear model too)

In [10]:
class ModelSettings:
    val_size = 0.1
    hidden_dim = 32
    optimizer = keras.optimizers.Adam(lr=0.001)
    batch_size = 16 # 16 for LSTM, 741 for linear (i.e. just use batch GD)
    epochs = 35 # 35 for LSTM, 250 for linear
    early_stop = EarlyStopping(monitor='loss', patience=10, verbose=1)
    loss = 'binary_crossentropy'
    metrics = ['accuracy']
    final_activation = 'sigmoid'

Instructions for updating:
Colocations handled automatically by placer.


### define cross validation + aggregation function

In [11]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

def train_model(X, Y, model_setup_func, kfold=True):
    if kfold:
        # define 10-fold cross validation test harness
        my_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
        cvscores = []
        for train, test in my_kfold.split(X, Y):
            model = model_setup_func(X, kfold)
            model.fit(X[train], Y[train], epochs=ModelSettings.epochs, batch_size=ModelSettings.batch_size,
                      validation_split=ModelSettings.val_size, verbose=0)
            # evaluate the model
            scores = model.evaluate(X[test], Y[test], verbose=0)
            print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
            cvscores.append(scores[1] * 100)
        print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
        
    else:
        model = model_setup_func(X, kfold)
        model.fit(X, Y, epochs=ModelSettings.epochs, batch_size=ModelSettings.batch_size,
                  validation_split=ModelSettings.val_size, callbacks=[ModelSettings.early_stop])
    return model, cvscores

### define linear model setup with corrmats

In [12]:
def setup_linear_model(Xcorr, kfold=True):
    model = Sequential()
    #model.add(Dropout(0.5, input_shape=Xcorr.shape[1:]))
    model.add(Dense(1, kernel_regularizer=L1L2(0.02, 0.02), bias_regularizer=L1L2(0.02, 0.02), 
                    input_shape=Xcorr.shape[1:], activation=ModelSettings.final_activation))
    model.compile(loss=ModelSettings.loss, optimizer=keras.optimizers.Adam(lr=0.0001), metrics=ModelSettings.metrics)
    if not kfold:
        print(model.summary())
    return model

### define LSTM setup with timeseries

In [13]:
def setup_lstm(X, kfold=True):
    model = Sequential()
    reg = L1L2(0.001, 0.001)
    model.add(LSTM(ModelSettings.hidden_dim, input_shape=X.shape[1:], dropout=0.5, recurrent_dropout=0.5, 
                   kernel_regularizer=reg, recurrent_regularizer=reg, bias_regularizer=reg))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation=ModelSettings.final_activation))
    model.compile(loss=ModelSettings.loss, optimizer=ModelSettings.optimizer, metrics=ModelSettings.metrics)
    if not kfold:
        print(model.summary())
    return model

### Linear model with 10-fold CV

In [14]:
model_lin, cvscores_lin = train_model(Xcorr, Y, setup_linear_model, kfold=True)

Instructions for updating:
Use tf.cast instead.
acc: 66.67%
acc: 73.33%
acc: 55.41%
acc: 56.76%
acc: 55.41%
acc: 54.05%
acc: 55.41%
acc: 58.11%
acc: 58.11%
acc: 56.16%
58.94% (+/- 5.85%)


### LSTM with 10-fold CV

In [15]:
model, cvscores = train_model(X, Y, setup_lstm, kfold=True)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
acc: 58.67%
acc: 57.33%
acc: 50.00%
acc: 47.30%
acc: 47.30%
acc: 50.00%
acc: 60.81%
acc: 52.70%
acc: 55.41%
acc: 54.79%
53.43% (+/- 4.50%)


### get top 10 most salient features of autism dx's in LSTM

In [92]:
# from https://github.com/experiencor/deep-viz-keras
from guided_backprop import GuidedBackprop

In [93]:
guided_bprop = GuidedBackprop(model)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/guided_backprop_ckpt


autism is the 0 class, therefore you need to find the most negative derivatives for those samples

In [ ]:
autX = X[Y == 0, :, :]
mask = np.zeros((autX.shape[1:]))
for i in range(autX.shape[0]):
    mask += guided_bprop.get_mask(autX[i, :, :])

mask /= autX.shape[0]
mask_1d = mask.mean(axis=0)

In [126]:
regiondf = pd.read_csv('CC200_ROI_labels.csv')

In [176]:
regiondf.iloc[mask_1d.argsort()[:10]]['AAL'].values

array(['["Cingulum_Ant_R": 0.44]["Cingulum_Ant_L": 0.40]',
       '["Cingulum_Mid_L": 0.32]["Cingulum_Mid_R": 0.22]["Cingulum_Ant_L": 0.21]["Cingulum_Ant_R": 0.20]',
       '["Cerebelum_Crus1_L": 0.75]["Cerebelum_6_L": 0.18]',
       '["Frontal_Mid_L": 0.72]["Frontal_Sup_L": 0.28]',
       '["Frontal_Sup_R": 0.41]["Supp_Motor_Area_R": 0.41]["Frontal_Sup_Medial_R": 0.18]',
       '["Precuneus_L": 0.46]["Precuneus_R": 0.36]',
       '["Frontal_Sup_R": 0.50]["Frontal_Sup_Medial_R": 0.42]',
       '["Fusiform_R": 0.54]["Hippocampus_R": 0.21]["ParaHippocampal_R": 0.19]',
       '["Cingulum_Ant_L": 0.66]["Frontal_Sup_Medial_L": 0.28]',
       '["Fusiform_L": 0.45]["Temporal_Inf_L": 0.21]["ParaHippocampal_L": 0.13]'],
      dtype=object)

#### renamed
ACC bilateral
MCC bilateral
Cerebellum left
Mid Frontal left
Sup Frontal left
Precuneus bilateral
Sup Frontal bilateral
Fusiform left
ACC left
Fusiform right

## Experimental

### LSTM with aux inputs

In [ ]:
# use functional api to create LSTM and merge output with auxiliary input
# TIME_SERIES --->  LSTM --->|
#                            MERGED ---> | --> diagnosis
#     PATIENT_FEATURES ---> |

reg = L1L2(0.001, 0.001)
channel1 = keras.layers.Input(shape=X.shape[1:], name='channel1')

model_gru = LSTM(ModelSettings.hidden_dim, dropout=0.5, recurrent_dropout=0.5,
                kernel_regularizer=reg, recurrent_regularizer=reg, bias_regularizer=reg)(channel1)

# At this point, we feed into the model our auxiliary input data by concatenating it with the GRU output
auxiliary_input = keras.layers.Input(shape=(offline_train.shape[1],), name='aux_input')
x = keras.layers.concatenate([model_gru, auxiliary_input])
x = Dropout(0.5)(x)  

# We stack a another layer on top
#x = Dense(ModelSettings.hidden_dim, activation='sigmoid')(x)
#x = Dropout(0.5)(x) 

# And finally we add the classification layer
main_output = Dense(1, activation=ModelSettings.final_activation, name='main_output')(x)

# now we compile and run
deep = keras.models.Model(inputs=[channel1, auxiliary_input], outputs=[main_output])
deep.compile(loss=ModelSettings.loss, optimizer=ModelSettings.optimizer, metrics=ModelSettings.metrics)
print(deep.summary())
deep.fit([X, offline_train], Y,
         epochs=ModelSettings.epochs, batch_size=ModelSettings.batch_size,
         validation_split=ModelSettings.val_size, callbacks=[ModelSettings.early_stop])


### temporal CNN

In [ ]:
reg = L1L2(0.0001, 0.0001)
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=X.shape[1:],
                kernel_regularizer=reg, bias_regularizer=reg))
model.add(Dropout(0.5))
model.add(AvgPool1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(AvgPool1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(AvgPool1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(AvgPool1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.5))

#model.add(Dense(16, activation='relu', kernel_regularizer=reg, bias_regularizer=reg))
model.add(Dense(1, activation=ModelSettings.final_activation))
model.compile(loss=ModelSettings.loss, optimizer=keras.optimizers.Adam(lr=0.002), metrics=ModelSettings.metrics)
print(model.summary())
model.fit(X, Y, epochs=ModelSettings.epochs, batch_size=ModelSettings.batch_size,
         validation_split=ModelSettings.val_size, callbacks=[ModelSettings.early_stop])